In [1]:
# Use this to check if java_home is correctly set
import subprocess
subprocess.call("ECHO $JAVA_HOME", shell=True)
# If JAVA_HOME is not set, then jpype will fail. 

/Users/gomeshe/Library/Java/JavaVirtualMachines/openjdk-20.0.1/Contents/Home


0

In [2]:
import jpype

import jpype.imports
from jpype.types import *

# Starts the JVM
jpype.startJVM()

In [3]:
# Add the moa jar to the class path
jpype.addClassPath('/Users/gomeshe/Dropbox/ciencia_computacao/dev/Using-MOA-API/moa.jar')

## Basic classification using ARF

Implements the ```test_train_loop``` for MOA and ```run_experiment``` helper functions for the analysis

SAVED THE RESULTS FROM THE EXPERIMENTS BECAUSE RIVER TAKES TOO LONG TO EXECUTE!! Especially ARF100


In [4]:
import pandas as pd
from river import stream
from river import metrics

## Function to abstract the test and train loop
def test_train_loop(stream, learner, evaluator, maxInstances=1000, sampleFrequency=100):
    instancesProcessed = 1
    
    learner.setModelContext(stream.getHeader())
    
    data = []
    performance_names = []
    performance_values = []
    
    while stream.hasMoreInstances() and instancesProcessed <= maxInstances:
        trainInst = stream.nextInstance()
        testInst = trainInst
    
        prediction = learner.getVotesForInstance(testInst)
    
        evaluator.addResult(testInst, prediction)
        learner.trainOnInstance(trainInst)
    
        if instancesProcessed == 1:
            performance_measurements = evaluator.getPerformanceMeasurements()
            performance_names = ["".join(measurement.getName()) for measurement in performance_measurements]
    
        if instancesProcessed % sampleFrequency == 0:
            performance_values = [measurement.getValue() for measurement in evaluator.getPerformanceMeasurements()]
            data.append(performance_values)
        
        instancesProcessed += 1
    
    return pd.DataFrame(data, columns=performance_names)


def run_experiment(model, dataset):
    accuracy = metrics.Accuracy()
    start = time.perf_counter()

    X, Y = dataset[:, :-1], dataset[:, -1]
    # Downsampling the data
    # Select every 10th element starting from the first element
    # Useful during development, when you are making sure your
    #    algorithm works before doing the actual experiments.
    # X, Y = X[::200], Y[::200]

    ds = stream.iter_array(X, Y)

    for (x, y) in ds:
        yp = model.predict_one(x)
        accuracy.update(y, yp)
        model.learn_one(x, y)

    return accuracy.get(), dt.timedelta(seconds=time.perf_counter() - start).seconds

# Experiments MOA vs River using ARF

## MOA ARF10

In [5]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf10 = AdaptiveRandomForest()
arf10.getOptions().setViaCLIString("-s 10")
arf10.setRandomSeed(1)
arf10.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf10, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 49 s, sys: 426 ms, total: 49.4 s
Wall time: 46.1 s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,89.800000,53.123707,61.797753,37.423313,98.805257,43.558282,NaN,NaN,NaN
1,2000.0,90.400000,58.706036,66.017699,43.026706,98.556825,50.148368,NaN,NaN,NaN
2,3000.0,90.500000,60.598715,66.470588,45.297505,98.507463,52.399232,NaN,NaN,NaN
3,4000.0,90.750000,61.863907,67.081851,46.298984,98.338870,54.281567,NaN,NaN,NaN
4,5000.0,91.000000,62.924343,67.811159,47.429907,98.334942,55.490654,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.923958,74.948497,69.319921,68.486717,96.429711,71.908980,59.043928,14.564831,89.870517
96,97000.0,83.809278,74.746905,69.173242,68.236793,96.439987,71.804000,58.749069,14.620730,89.712358
97,98000.0,83.718367,74.578505,69.067928,68.023407,96.465770,71.732947,58.511980,14.578005,89.524733
98,99000.0,83.627273,74.403858,68.943516,67.800954,96.489930,71.634394,58.273531,14.432990,89.375391


CPU times: user 48.2 s, sys: 193 ms, total: 48.4 s
Wall time: 47.4 s

83.527000

## RIVER ARF10

In [4]:
%%time
from river.forest import ARFClassifier

import time
import datetime as dt
import pandas as pd
import os

rtg_2abrupt_path = '/Users/gomeshe/Desktop/data/RTG_2abrupt.csv'

river_arf10 = ARFClassifier(
    n_models=10,
    max_features=0.60
)

river_rtg_2abrupt = pd.read_csv(rtg_2abrupt_path).to_numpy()

result = run_experiment(river_arf10, river_rtg_2abrupt)
print('Learner: ', river_arf10, ' Result: ', result)

Learner:  ARFClassifier  Result:  (0.79017, 379)
CPU times: user 6min 17s, sys: 8.05 s, total: 6min 25s
Wall time: 6min 21s


CPU times: user 6min 5s, sys: 3.75 s, total: 6min 8s
Wall time: 6min 9s

Learner:  ARFClassifier  Result:  (0.80196, 368)

## MOA ARF30

In [28]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf30 = AdaptiveRandomForest()
arf30.getOptions().setViaCLIString("-s 30")
arf30.setRandomSeed(1)
arf30.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf30, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 2min 40s, sys: 508 ms, total: 2min 40s
Wall time: 2min 36s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,47.856968,58.426966,31.901840,98.685783,38.650307,NaN,NaN,NaN
1,2000.0,89.550000,54.067347,63.008850,37.982196,98.496693,45.400593,NaN,NaN,NaN
2,3000.0,89.500000,55.650876,62.941176,39.539347,98.305768,47.600768,NaN,NaN,NaN
3,4000.0,90.050000,58.071433,64.590747,42.235123,98.399275,49.927431,NaN,NaN,NaN
4,5000.0,90.160000,58.288796,64.806867,42.523364,98.455598,50.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,84.034375,75.052102,69.530644,68.703163,96.978332,70.715536,58.354866,17.525163,90.082503
96,97000.0,83.921649,74.852764,69.387194,68.457245,96.988813,70.620877,58.025742,17.515924,89.952536
97,98000.0,83.835714,74.693075,69.290865,68.253873,97.006299,70.590061,57.839428,17.362887,89.764364
98,99000.0,83.754545,74.535105,69.184933,68.051251,97.024250,70.551270,57.618305,17.163555,89.620224


CPU times: user 2min 40s, sys: 508 ms, total: 2min 40s
Wall time: 2min 36s

83.666000

## RIVER ARF30

In [30]:
%%time

river_arf30 = ARFClassifier(
    n_models=30,
    max_features=0.60
)

river_rtg_2abrupt = pd.read_csv(rtg_2abrupt_path).to_numpy()

result = run_experiment(river_arf30, river_rtg_2abrupt)
print('Learner: ', river_arf30, ' Result: ', result)

Learner:  ARFClassifier  Result:  (0.80162, 1145)
CPU times: user 18min 56s, sys: 11.7 s, total: 19min 7s
Wall time: 19min 6s


CPU times: user 18min 56s, sys: 11.7 s, total: 19min 7s
Wall time: 19min 6s

Learner:  ARFClassifier  Result:  (0.80162, 1145)

## ARF100

In [31]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 11min 15s, sys: 2.25 s, total: 11min 17s
Wall time: 10min 28s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


CPU times: user 11min 15s, sys: 2.25 s, total: 11min 17s
Wall time: 10min 28s

83.616000

## RIVER ARF100

In [32]:
%%time

river_arf100 = ARFClassifier(
    n_models=100,
    max_features=0.60
)

river_rtg_2abrupt = pd.read_csv(rtg_2abrupt_path).to_numpy()

result = run_experiment(river_arf100, river_rtg_2abrupt)
print('Learner: ', river_arf100, ' Result: ', result)

Learner:  ARFClassifier  Result:  (0.80145, 4384)
CPU times: user 1h 13min 34s, sys: 42.7 s, total: 1h 14min 17s
Wall time: 2h 5s


CPU times: user 1h 13min 34s, sys: 42.7 s, total: 1h 14min 17s
Wall time: 2h 5s

Learner:  ARFClassifier  Result:  (0.80145, 4384)

## ARF100 -j 4

In [8]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 4")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 8min 25s, sys: 8.7 s, total: 8min 34s
Wall time: 3min 45s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


## ARF100 -j 8

In [7]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 8")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 9min 43s, sys: 12.7 s, total: 9min 55s
Wall time: 4min 18s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


CPU times: user 9min 43s, sys: 12.7 s, total: 9min 55s
Wall time: 4min 18s

83.616000	

## ARF100 -j 2

In [8]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 2")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 9min 42s, sys: 8.26 s, total: 9min 50s
Wall time: 6min 20s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


CPU times: user 9min 42s, sys: 8.26 s, total: 9min 50s
Wall time: 6min 20s

83.616000

## ARF100 -j 5

In [9]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 5")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 9min 39s, sys: 8.91 s, total: 9min 48s
Wall time: 4min 37s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473
